In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import json
import logging

logging.basicConfig(level=logging.WARNING)

from data.VulnerabilityReport import create_from_flama_json # this is the function that creates the VulnerabilityReport object from the json data
from ai.LLM.LLMServiceStrategy import LLMServiceStrategy 

In [3]:
from pathlib import Path

data_folder = Path('../../data/')
# We need the json data, so we are loading it from the file
with open(data_folder / Path('Dataset.json')) as f:
    data = json.load(f)

# Choose your Strategy
Right now, we support two strategies listed below.
Execute the one you like more :)

In [4]:
my_strategy = None

## Option 1) OpenAI - Remote
Uses OpenAIs GPT-4o by default.

Relatively cheap and quite good.


In [ ]:
from ai.LLM.Stretegies.OpenAIService import OpenAIService

my_strategy = OpenAIService()

## Option 2) Anthropic - Remote
Uses Anthropics Claude 3 Opus by default.

Almost perfect but quite expensive.


In [ ]:
from ai.LLM.Stretegies.OpenAIService import OpenAIService

my_strategy = OpenAIService()

## Option 3) OLLAMA - Local
Here, you can choose the model you want to use. If you don't know which one to choose, you can just leave it empty and the default model will be used. Available models can be found [here](https://ollama.com/library).

The performance (both in time and quality) depends heavily on the model used.

In [5]:
from ai.LLM.Stretegies.OLLAMAService import OLLAMAService # this is the service that uses ollama to generate solution

model_name=None # but can be anything from here https://ollama.com/library
my_strategy = OLLAMAService(model_name=model_name)

# LLM Service
This is the service that uses the strategy you chose. It is the top abstraction layer that will be used to generate solutions.

In [6]:
llm_service = LLMServiceStrategy(my_strategy)

# Data Setup
We instance a vulnerability report object and add findings to it. We also sort the findings by severity.
This object takes the LLM service and acts as main abstraction layer for the data.


In [7]:
n=5 # number of findings to import
vulnerability_report = create_from_flama_json(data, n = n, llm_service=llm_service) # here, we create the VulnerabilityReport object, consisting of a list of Finding objects which each have Solution objects. We pass the llm_service to the VulnerabilityReport object, but this can also just be omitted, in which case it will be created in each Finding object.
vulnerability_report.sort() # this will sort the findings by severity

# Lets get going with AI - 🚀🌕

## First: We add categories to the Findings
This will help us later to improve prompts (hopefully)

In [8]:
vulnerability_report.add_category() # this will add categories to the findings

100%|██████████| 5/5 [00:04<00:00,  1.09it/s]


## Then we add solutions 🎉
The function below adds long and short solutions to the findings. It also adds keywords for future research.

For this function, we have multiple layers that may catch errors. So if you see a warning but no error afterwards, the waring was caught. Only if you see an error, data wasn't generated correctly.

In [ ]:
vulnerability_report.add_solution(long=True, short=True, search_term=True) # this will add solutions to the findings

 80%|████████  | 4/5 [00:41<00:10, 10.28s/it]

# Last but not least: Results
We save and display results in the coming cells.

In [ ]:
with open(data_folder / Path(f'VulnerabilityReport_{n if n !=-1 else "all"}_' + my_strategy.get_model_name().replace(":","_") +'.json'), 'w') as f:
    f.write(str(json.dumps(vulnerability_report.to_dict())))

In [ ]:
from IPython.display import HTML, Markdown
Markdown(vulnerability_report.to_html(table=True))


In [ ]:
HTML(str(vulnerability_report).replace('\n', '<br />'))
